# Testing proj, gdal and `cartopy` transformations

Tests:

- check that packages return correct CRS for an EPSG
- check that correct coordinates are returned

In [1]:
import os
from warnings import warn

from pprint import pprint

from pyproj import CRS, Transformer
import cartopy.crs as ccrs
from osgeo import osr

from numpy.testing import assert_array_almost_equal

import cartopy
import pyproj
import osgeo

from constant import NSIDC_EPSG_CODES, WGS84_EPSG, NPS_GRID_CORNERS, NPS_MAPX_LATLON

print(f"cartopy: {cartopy.__version__}")
print(f"pyproj: {pyproj.__version__}")
print(f"osgeo: {osgeo.__version__}")
print()
print(f"EPSG Database version: {pyproj.database.get_database_metadata('EPSG.VERSION')}")
print(f"EPSG Database date: {pyproj.database.get_database_metadata('EPSG.DATE')}")

# Check that OSR_USE_NON_DEPRECATED
osr_use_non_deprecated = os.environ.get('OSR_USE_NON_DEPRECATED')
print(f"GDAL environment variable OSR_USE_NON_DEPRECATED is {osr_use_non_deprecated or 'not set'}")
if osr_use_non_deprecated != "NO":
     warn(" This will result in suggested replacement CRS being used!\n"
          "   If this is not the behavior you want, set OSR_USE_NON_DEPRECATED=NO")

ModuleNotFoundError: No module named 'osgeo'

In [3]:
os.environ['OSR_USE_NON_DEPRECATED'] = "YES"  # Set this to test updated deprecation
print(f"OSR_USE_NON_DEPRECATED is not set to {os.environ['OSR_USE_NON_DEPRECATED']}")

OSR_USE_NON_DEPRECATED is not set to YES


Test that correct CRS returned for EPSG code

In [4]:
def test_correct_crs_from_proj(epsg, target_crs_name):
    """Checks that CRS name returned from proj CRS 
    instantiation matches expected CRS name"""
    crs = CRS.from_epsg(epsg)
    result = crs.name
    assert result == target_crs_name, f"For EPSG:{epsg}: expected {target_crs_name}, got {result}" 

def test_correct_crs_from_osgeo(epsg, target_crs_name):
    """Checks that CRS name returned from osgeo.osr CRS 
    instantiation matches expected CRS name"""
    inSpatialRef = osr.SpatialReference()
    inSpatialRef.ImportFromEPSG(epsg)
    result = inSpatialRef.GetName()
    assert result == target_crs_name, f"For EPSG:{epsg}: expected {target_crs_name}, got {result}" 

def test_correct_crs_from_cartopy(epsg, target_crs_name):
    """Checks that CRS name returned from cartopy CRS 
    instantiation matches expected CRS name"""
    crs = ccrs.epsg(epsg)
    result = crs.name
    assert result == target_crs_name, f"For EPSG:{epsg}: expected {target_crs_name}, got {result}" 

In [5]:
for epsg, target_crs_name in NSIDC_EPSG_CODES.items():
    print(f"Testing EPSG:{epsg} {target_crs_name}")
    test_correct_crs_from_proj(epsg, target_crs_name)
    test_correct_crs_from_osgeo(epsg, target_crs_name)
    test_correct_crs_from_cartopy(epsg, target_crs_name)

Testing EPSG:3408 NSIDC EASE-Grid North
Testing EPSG:3409 NSIDC EASE-Grid South
Testing EPSG:3410 NSIDC EASE-Grid Global
Testing EPSG:6931 WGS 84 / NSIDC EASE-Grid 2.0 North
Testing EPSG:6932 WGS 84 / NSIDC EASE-Grid 2.0 South
Testing EPSG:6933 WGS 84 / NSIDC EASE-Grid 2.0 Global
Testing EPSG:3411 NSIDC Sea Ice Polar Stereographic North
Testing EPSG:3412 NSIDC Sea Ice Polar Stereographic South
Testing EPSG:3413 WGS 84 / NSIDC Sea Ice Polar Stereographic North
Testing EPSG:3976 WGS 84 / NSIDC Sea Ice Polar Stereographic South


/home/apbarret/mambaforge/envs/epsg_io_spoof/lib/python3.11/site-packages/osgeo/osr.py:385: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


## Proj

In [6]:
def proj_transform_points(points, source_epsg, target_epsg, direction='FORWARD'):
    source_crs = CRS.from_epsg(source_epsg)
    target_crs = CRS.from_epsg(target_epsg)
    transformer = Transformer.from_crs(source_crs, target_crs)
    x, y = zip(*points)
    result = transformer.transform(x, y)
    return list(zip(*result))

In [7]:
import numpy as np

corners_ll = proj_transform_points(NPS_GRID_CORNERS, 3411, 4326)
assert_array_almost_equal(np.array(corners_ll), np.array(NPS_MAPX_LATLON), decimal=6)

In [ ]:
NPS_MAPX_LATLON

## Cartopy

Define cartopy crs for source

In [ ]:
def make_projected_crs(epsg_code):
    try:
        crs = ccrs.epsg(epsg_code)
    except Exception as err:
        raise err
    return crs

def make_geodetic_crs(epsg_code):
    try:
        crs = ccrs.Geodetic()
    except Exception as err:
        raise err
    return crs
        
def make_cartopy_crs(epsg_code):
    if not isinstance(epsg_code, int):
        raise TypeError("Expect EPSG code to be integer, e.g. 4326")

    try:
        crs = make_projected_crs(epsg_code)
    except ValueError as err:
        if err.__str__() == "EPSG code does not define a projection":
            crs = make_geodetic_crs(epsg_code)
    except Exception as err:
        raise err
    return crs
        
def cartopy_transform_points(points, source_epsg, target_epsg):
    source_crs = make_cartopy_crs(source_epsg)
    target_crs = make_cartopy_crs(target_epsg)

    x, y = zip(*nps_grid_corners)
    x = np.array(x)
    y = np.array(y)
    result = target_crs.transform_points(source_crs, x, y)
    return [(lat, lon) for lat, lon in result[:, [1,0]]]  # Assumes axis order 

In [ ]:
cartopy_transform_points(nps_grid_corners, 3411, 4326)

In [ ]:
def make_cartopy_crs_from_scratch(epsg_code):
    """Makes a cartopy crs using proj strings.  proj strings are returned from proj CRS
    This is done because defining CRS from epsg does no create correct bounds"""
    proj_params = CRS.from_epsg(epsg_code).to_dict()
    
    proj2ctopy = {'a': 'semimajor_axis', 'b': 'semiminor_axis'}  # Converts proj parameter names to cartopy Globe keywords

    # Make the globe instance
    globe = ccrs.Globe(**{**{'ellipse': None}, **{proj2ctopy[k]: v for k, v in proj_params.items() if k in ['a', 'b']}})  # Must set ellipse explicitly to None otherwise is wgs84
    crs = ccrs.Projection({k: v for k, v in proj_params.items() if k not in ['a', 'b']}, globe=globe)
    return crs

In [ ]:
make_cartopy_crs_from_scratch(3411)

In [ ]:
for ctpy, proj in zip(ctpy_dd_corners, proj_dd_corners):
    assert ctpy == proj

## GDAL

See [Python bindings](https://gdal.org/tutorials/osr_api_tut.html#python-bindings) for `osgeo`

In [ ]:
from osgeo import osr

In [ ]:
def get_osgeo_crs(epsg):
    """Returns CRS from EPSG using osgeo osr"""
    crs = osr.SpatialReference()
    crs.ImportFromEPSG(epsg)
    return crs

def osgeo_transform_points(points, source_epsg, target_epsg):
    source_crs = get_osgeo_crs(source_epsg)
    target_crs = get_osgeo_crs(target_epsg)
    transformer = osr.CoordinateTransformation(source_crs, target_crs)
    result = transformer.TransformPoints(points)
    return [(lat, lon) for lat, lon, z in result]

In [ ]:
gdal_dd_corners = osgeo_transform_points(NPS_GRID_CORNERS, 3411, WGS84_EPSG)

In [ ]:
for gdal, proj in zip(gdal_dd_corners, proj_dd_corners):
    gdal = (gdal[0], gdal[1])
    assert gdal == proj, f"{gdal} != {proj}"

In [ ]:
for gdal, ctpy in zip(gdal_dd_corners, ctpy_dd_corners):
    gdal = (gdal[0], gdal[1])
    assert gdal == ctpy, f"{gdal} != {ctpy}"